# core

> Fill in a module description here

In [ ]:
#| default_exp core

# Packages

In [ ]:
from abc import ABC, abstractmethod

import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors 

from collections import Counter
from collections import defaultdict

from joblib import Parallel, delayed, dump, load
import ipdb

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()